In [35]:
# step11 : 가변 길이 인수(순전파)
import numpy as np

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x


## 수정사항 ##

class Function:
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,)  # 튜플 형태 반환

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [36]:
# step12 : 가변 길이 인수(개선)
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator)

def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x
# 첫 번째 개선: 함수를 사용하기 쉽게
# 두 번째 개선: 함수 구현이 쉽게 
class Function:
  def __call__(self, *inputs):  # 1.1: * 가변 길이 인수
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)  # 2.1: 별표 붙여 언팩
    # 2.2: 튜플 아니면 추가 지원
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    # 1.2: 리스트 원소가 하나면 첫 번째 원소 반환
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

# Add class 생성 과정 감춰지도록
def add(x0, x1):
    return Add()(x0, x1)

x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


In [37]:
# step13 : 가변 길이 인수(역전파)
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs] # outputs의 미분값들을 리스트에
      gxs = f.backward(*gys)  # f의 역전파 호출 (리스트 언팩)
      if not isinstance(gxs, tuple):  # gxs 튜플 아닐 경우 튜플로
        gxs = (gxs,)
        
      for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분값을 grad에 저장
        # f.inputs[i]의 미분값은 gxs[i]에 대응
        x.grad = gx
        
        if x.creator is not None: 
          funcs. append(x.creator)


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y
  
  # add의 연적파는 미분값 그대로
  def backward(self, gy):
    return gy, gy

def add(x0, x1):
    return Add()(x0, x1)


class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data  # input -> inputs[0]로 변동
    gx = 2 * x * gy
    return gx

def square(x):
    return Square()(x)

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


In [38]:
#step14 : 같은 변수 반복 사용 문제 해결
class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은 지원하지 않습니다'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func

    # 미분값 초기화
  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs] 
      gxs = f.backward(*gys) 
      if not isinstance(gxs, tuple):
        gxs = (gxs,)
        
      for x, gx in zip(f.inputs, gxs): 
        # 출력 쪽에서 전해지는 미분값 그대로 대입하므로 같은 변수 사용 시 덮어써지는 문제
        # x.grad = gx
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx  # 덮어쓰는 것이 아닌 미분값을 더해주도록     

        if x.creator is not None: 
          funcs.append(x.creator)

# 첫 번째 계산
x=Variable(np.array(3.0))
y=add(x, x)
y.backward()
print(x.grad)

# 두 번째 계산(같은 x 를 사용하여 다른 계산을 수행)
x.cleargrad()
y=add(add(x, x), x) 
y.backward() 
print(x.grad)

2.0
3.0


In [39]:
#step16 : 복잡한 계산 그래프
import numpy as np


class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수 기록 변수

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대 기록

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    ## ----- 수정 부분 -----
    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        # 세대 순 정렬
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
    ## ---------------------

    while funcs:
      f = funcs.pop()
      gys = [output.grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) # 기존: funcs.append(x.creator)


def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x


class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    # generation 입력 변수 중 젤 큰 generation
    self.generation = max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    self.outputs = outputs
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx


def square(x):
  return Square()(x)


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy


def add(x0, x1):
  return Add()(x0, x1)


x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


In [40]:
#step17 :메모리 관리와 순환 참조
import weakref

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      # 수정 전 : gys = [output.grad for output in f.outputs]
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) 


def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x


class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    self.generation = max([x.generation for x in inputs])
    for output in outputs:
      output.set_creator(self)
    self.inputs = inputs
    ## 수정사항
    self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()

# 동작 확인
for i in range(10):
  x = Variable(np.random.randn(10000)) # 거대 데이터 
  y= square(square(square(x)))

In [51]:
#step18 : 메모리 절약 모드
# 1. 필요없는 미분 값 제거
# 2. 역전파 필요 없는 경우용 모드 제공
import contextlib

class Config:
  # 역전파 가능 여부 확인
  enable_backprop = True

# with 문을 사용한 모드 전환 시 contextlib 사용 
@contextlib.contextmanager
def using_config(name, value):  # name: Config 이름
  # 전처리
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    # 후처리
    setattr(Config, name, old_value)

def no_grad():
  return using_config('enable_backprop', False)

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 

  def cleargrad(self):
    self.grad = None

  def backward(self, retain_grad=False): # 인자 추가
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) 
      
      # 코드 추가
      if not retain_grad:
        for y in f.outputs:
          y().grad = None # y는 약한 참조


def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return x


class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 * x * gy
    return gx


def square(x):
  return Square()(x)


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy


def add(x0, x1):
  return Add()(x0, x1)


x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad) 
print(x0.grad, x1.grad)

with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)

None None
2.0 1.0


In [60]:
#step19 : 변수 사용성 개선
class Variable:
  def __init__(self, data, name=None):  # name: 변수 이름 지정
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.name = name  # 추가
    self.grad = None
    self.creator = None
    self.generation = 0 

  @property # shape 메서드를 인스턴스 변수처럼 사용 (x.shape() 대신 x.shape)
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):  # len() 함수 구현
    return len(self.data)
  
  def __repr__(self): # print 구현
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      # 수정 전 : gys = [output.grad for output in f.outputs]
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) 


x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
x.name = 'x'

print(x.name)
print(x.shape)
print(x)

x
(2, 3)
variable([[1 2 3]
          [4 5 6]])


In [63]:
#step20 : 연산자 오버로드(1)
class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy


def add(x0, x1):
  return Add()(x0, x1)


class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0


def mul(x0, x1):
  return Mul()(x0, x1)


Variable.__add__ = add
Variable.__mul__ = mul

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [70]:
#step21 : 연산자 오버로드(2)
# Variable, ndarray, int, float 함께 쓸 수 있도록

class Variable:
  __array_priority__ = 200  # 연산자 우선순위: Variable > ndarray 
  def __init__(self, data, name=None): 
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.name = name 
    self.grad = None
    self.creator = None
    self.generation = 0 

  @property 
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self): 
    return len(self.data)
  
  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 

  def cleargrad(self):
    self.grad = None

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator) 

# ndarray를 Variable로 변환 
def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  # Variable이 아니면 Variable로 변환
  return Variable(obj)

class Function:
  def __call__(self, *inputs):
    inputs = [as_variable(x) for x in inputs] # 코드 추가 (ndarray -> Variable)

    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    return gy, gy


def add(x0, x1):
  x1 = as_array(x1) # 코드 추가 (float, int -> ndarray)
  return Add()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add # 좌우 인자 바뀌어도 똑같게
Variable.__mul__ = mul
Variable.__rmul__ = mul # 좌우 인자 바뀌어도 똑같게


x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

variable(5.0)
variable(5.0)


In [71]:
#step22 : 연산자 오버로드(3)
import weakref
import numpy as np
import contextlib


class Config:
  enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, old_value)


def no_grad():
  return using_config('enable_backprop', False)


class Variable:
  __array_priority__ = 200

  def __init__(self, data, name=None):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.name = name
    self.grad = None
    self.creator = None
    self.generation = 0

  @property
  def shape(self):
    return self.data.shape

  @property
  def ndim(self):
    return self.data.ndim

  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

  def __len__(self):
    return len(self.data)

  def __repr__(self):
    if self.data is None:
      return 'variable(None)'
    p = str(self.data).replace('\n', '\n' + ' ' * 9)
    return 'variable(' + p + ')'

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  def cleargrad(self):
    self.grad = None

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)

    while funcs:
      f = funcs.pop()
      gys = [output().grad for output in f.outputs]  # output is weakref
      gxs = f.backward(*gys)
      if not isinstance(gxs, tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      if not retain_grad:
        for y in f.outputs:
          y().grad = None  # y is weakref


def as_variable(obj):
  if isinstance(obj, Variable):
      return obj
  return Variable(obj)


def as_array(x):
  if np.isscalar(x):
      return np.array(x)
  return x


class Function:
  def __call__(self, *inputs):
      inputs = [as_variable(x) for x in inputs]

      xs = [x.data for x in inputs]
      ys = self.forward(*xs)
      if not isinstance(ys, tuple):
          ys = (ys,)
      outputs = [Variable(as_array(y)) for y in ys]

      if Config.enable_backprop:
          self.generation = max([x.generation for x in inputs])
          for output in outputs:
              output.set_creator(self)
          self.inputs = inputs
          self.outputs = [weakref.ref(output) for output in outputs]

      return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
      raise NotImplementedError()

  def backward(self, gys):
      raise NotImplementedError()


class Add(Function):
  def forward(self, x0, x1):
      y = x0 + x1
      return y

  def backward(self, gy):
      return gy, gy


def add(x0, x1):
  x1 = as_array(x1)
  return Add()(x0, x1)


class Mul(Function):
  def forward(self, x0, x1):
      y = x0 * x1
      return y

  def backward(self, gy):
      x0, x1 = self.inputs[0].data, self.inputs[1].data
      return gy * x1, gy * x0


def mul(x0, x1):
  x1 = as_array(x1)
  return Mul()(x0, x1)


class Neg(Function):
  def forward(self, x):
    return -x

  def backward(self, gy):
    return -gy


def neg(x):
  return Neg()(x)


class Sub(Function):
  def forward(self, x0, x1):
    y = x0 - x1
    return y

  def backward(self, gy):
    return gy, -gy


def sub(x0, x1):
  x1 = as_array(x1)
  return Sub()(x0, x1)


def rsub(x0, x1):
  x1 = as_array(x1)
  return sub(x1, x0)


class Div(Function):
  def forward(self, x0, x1):
    y = x0 / x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    gx0 = gy / x1
    gx1 = gy * (-x0 / x1 ** 2)
    return gx0, gx1


def div(x0, x1):
  x1 = as_array(x1)
  return Div()(x0, x1)


def rdiv(x0, x1):
  x1 = as_array(x1)
  return div(x1, x0)


class Pow(Function):
  def __init__(self, c):
    self.c = c

  def forward(self, x):
    y = x ** self.c
    return y

  def backward(self, gy):
    x = self.inputs[0].data
    c = self.c

    gx = c * x ** (c - 1) * gy
    return gx


def pow(x, c):
  return Pow(c)(x)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

x = Variable(np.array(2.0))
y = -x
print(y)  # variable(-2.0)

y1 = 2.0 - x
y2 = x - 1.0
print(y1)  # variable(0.0)
print(y2)  # variable(1.0)

y = 3.0 / x
print(y)  # variable(1.5)

y = x ** 3
y.backward()
print(y)  # variable(8.0)

variable(-2.0)
variable(0.0)
variable(1.0)
variable(1.5)
variable(8.0)
